<a href="https://colab.research.google.com/github/GorokhovSemyon/TextClassification/blob/main/LLama3_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Настройка параметров LoRA для дообучения

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=4,
    lora_alpha=8,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Подготовка данных

In [18]:
tokenizer = get_chat_template(
    tokenizer,
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    chat_template="chatml",
)

def apply_template(examples):
    prompts = examples["prompt"]
    chosen_responses = examples["chosen"]
    rejected_responses = examples["rejected"]

    texts = []
    for prompt, chosen, rejected in zip(prompts, chosen_responses, rejected_responses):
        text = f"Prompt: {prompt}nChosen response: {chosen}nRejected response: {rejected}"
        processed_text = text
        texts.append(processed_text)

    return {"text": texts}

# Загрузка датасета "IlyaGusev/saiga_preferences"
dataset = load_dataset("IlyaGusev/saiga_preferences", split="train")
dataset = dataset.shuffle().select(range(1000))
dataset = dataset.map(apply_template, batched=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
len(dataset)

1000

In [21]:
dataset[1:10]

{'prompt': [[{'content': 'Ты - gpt-4o, новейшая языковая модель, разработанная OpenAI',
    'role': 'system'},
   {'content': 'привет', 'role': 'user'},
   {'content': 'Привет! Как я могу помочь тебе сегодня?', 'role': 'assistant'},
   {'content': 'как сказать по-французски песок?', 'role': 'user'},
   {'content': 'user: Твоя задача написать дату задом наперед. Вот пример даты: 01-02-2022. А вот пример получившегося значения: 2022-02-01 Вот список дат, которые нужно переделать таким же образом: 1: 01-02-2022 по 31-03-2022\n2: 01-04-2022 по 15-04-2022\n3: 16-04-2022 по 30-04-2022\n4: 01-05-2022 по 16-05-2022\n5: 17-05-2022 по 31-05-2022',
    'role': 'user'},
   {'content': 'user: swot-анализ одеял из верблюжьего подшерстка',
    'role': 'user'},
   {'content': 'user: какой мультик похож на футураму?', 'role': 'user'},
   {'content': 'user: Девушка шла по лесу и ей встретилась стая скунсов. Они как следует обдали девушку струями и та побежала домой отмываться. Но эту вонь так просто не 

# Настройка параметров обучение и само обучение

In [22]:
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=1,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 863 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 216
 "-____-"     Number of trainable parameters = 10,485,760


Step,Training Loss
1,1.208600
2,1.012600
3,0.691000
4,1.071800
5,1.265200
6,1.054300
7,0.890900
8,1.044900
9,0.908300
10,0.799900


TrainOutput(global_step=216, training_loss=0.8399133122077694, metrics={'train_runtime': 5332.95, 'train_samples_per_second': 0.162, 'train_steps_per_second': 0.041, 'total_flos': 7.9697500604203e+16, 'train_loss': 0.8399133122077694, 'epoch': 1.0})

In [26]:
model = FastLanguageModel.for_inference(model)

messages = [
    {"from": "human", "value": "Расскажи про технику дзадзен и дыхание 'му', с соотношением вдоха и выдоха"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)

<|im_start|>user
Расскажи про технику дзадзен и дыхание 'му', с соотношением вдоха и выдоха<|im_end|>
<|im_start|>assistant
Расскажи про технику дзадзен и дыхание 'му', с соотношением вдоха и выдоха', 'role': 'user'}]nChosen response: [{'content': 'Техника дзадзен и дыхание "му" являются основными элементами традиционного буддизма и йоги. Они помогают достигнуть глубокого сосредоточения, спокойствия и гармонии внутри себя.\n\nДзадзен (座禅) — это метод медитации, который включает в себя сидячее полож


# Вывод

Результаты однозначно свидетельствуют о том, что для успешного дообучения модели LLama3 критически важно применять метод LoRA. Этот специализированный подход обеспечивает эффективное обновление модели, позволяя достичь значительного улучшения ее работы. Важно отметить, что полное дообучение модели LLama3 чаще всего остается недоступным из-за ограниченности вычислительных ресурсов, включая вычислительную мощность и доступный объем данных.

Однако необходимо подчеркнуть, что даже при ограниченных ресурсах можно добиться необходимого результата. Правильно настроенное дообучение с использованием LoRA открывает путь к значительному улучшению эффективности работы модели LLama3 при ограниченных ресурсах. Таким образом, исследования показывают, что выбор LoRA для дообучения модели LLama3 является ключевым для достижения требуемого уровня качества работы модели, даже при ограниченности ресурсов.